# Курсовая работа

Импорт библиотек

In [4]:
import re, math, codecs, os

In [5]:
from pprint import pprint
import numpy as np

In [6]:
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec

Подключение модели

In [7]:
m = "to.bin"

In [8]:
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True, unicode_errors='ignore')

Считывание предложений из файла

In [9]:
def cos(vA, vB):
    cos = np.dot(vA, vB) / (np.sqrt(np.dot(vA,vA)) * np.sqrt(np.dot(vB,vB)))
    return cos

In [10]:
info = []
words = []
final = []
with open('sentences.txt') as fh:
    sentences = fh.readlines()
for sentence in sentences:
    sentence.strip()
    sentence = sentence.split()
    count = 0
    for word in sentence:
        info.append(word)
        if len(info) == 3:
            words.append(info)
            info = []
    final.append(words)
    words = []

## Подсчет близости слов справа и слева

In [11]:
similar_words = []
for sentence in final:
    left_list = []
    best = -100
    ind = 0
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1 # Индикатор было ли то есть
            continue
        if not ind:
            pass
            try:
                left_list.append(model.wv[word[1]]) # Собираем список векторов левого контекста
            except Exception as e:
                print(e)
        elif ind:
            try:
                right = model.wv[word[1]]
                for vector in left_list:
                    best = max(best, cos(vector, right)) # Максимальное значение косинусной близости пар слов
            except Exception as e:
                continue
#                print(e)
    similar_words.append(best)

"word 'летесть' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'усмановна' not in vocabulary"
"word 'шурупский' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'тераэлектрон' not in vocabulary"
"word 'брейвик' not in vocabulary"
"word 'сабетта' not in vocabulary"
"word 'барбей' not in vocabulary"
"word 'натрехмерный' not in vocabulary"
"word 'доносилва' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'обывателиус' not in vocabulary"
"word 'пушкарная' not in vocabulary"
"word 'губпросвет' not in vocabulary"
"word 'вучик' not in vocabulary"
"word 'троллят' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'менгхин' not in vocabulary"
"word 'полож' not in vocabulary"
"word 'пильганова' not in vocabulary"
"word 'фейстоп' not in vocabulary"
"word 'экзотоксикант' not

In [75]:
with open('word_sim.txt', 'w') as fw:
    for i in similar_words:
        fw.write(str(i))
        fw.write('\n')

In [76]:
similar_words[:5]

[1.0000001, 0.26468626, 0.60758865, 0.62836206, 1.0000001]

## Кусочек кода для подсчета близости контекстов справа и слева

In [12]:
sentence_counts = []
for sentence in final:
    left_sum = 0
    right_sum = 0
    count_l = 0
    count_r = 0
    ind = 0
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1 # Индикатор было ли то есть
            continue
        if not ind:
            pass
            try:
                count_l += 1
                left_sum += model.wv[word[1]] # Складываем слова слева
            except Exception as e:
                print(e)
        elif ind:
            try:
                count_r += 1
                right_sum += model.wv[word[1]] # Складываем слова справа
            except Exception as e:
                continue
#                print(e)
    if count_l != 0 and count_r != 0:
        count = (left_sum / count_l, right_sum / count_l)
        sentence_counts.append(count)
    else:
        count = (left_sum, right_sum)
        sentence_counts.append(count)        

"word 'летесть' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'усмановна' not in vocabulary"
"word 'шурупский' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'тераэлектрон' not in vocabulary"
"word 'брейвик' not in vocabulary"
"word 'сабетта' not in vocabulary"
"word 'барбей' not in vocabulary"
"word 'натрехмерный' not in vocabulary"
"word 'доносилва' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'обывателиус' not in vocabulary"
"word 'пушкарная' not in vocabulary"
"word 'губпросвет' not in vocabulary"
"word 'вучик' not in vocabulary"
"word 'троллят' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'менгхин' not in vocabulary"
"word 'полож' not in vocabulary"
"word 'пильганова' not in vocabulary"
"word 'фейстоп' not in vocabulary"
"word 'экзотоксикант' not

In [13]:
count = 1
sim = []
for i in sentence_counts:
    sim.append([count, cos(i[0], i[1])])
    count += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [14]:
# Есть случаи, когда один из контекстов не определен
for i in sim:
    if isinstance(i[1], np.ndarray):
        i[1] = 0

In [15]:
print(len(sim))
sorted_sim = sorted(sim, key=lambda x: x[1], reverse=True)

1000


In [16]:
# Запишем все в файл, чтобы работать с ним на локальном компьютере
with open('part_sim.txt', 'w') as fw:
    for i in sorted_sim:
        a, b = i
        fw.write(str(a) + ' ' + str(b) + '\n')

## Средние векторы по каждому типу значения

In [17]:
# Вектор - сумма векторов слов
context = [sum(i) for i in sentence_counts]

In [18]:
# Скачиваем разметку
with open('Разметка.txt') as fh:
    tags = list(map(lambda l: l.strip(), fh.readlines()))
print(len(tags))

1000


In [19]:
from collections import Counter

In [20]:
unique = Counter(tags)
unique

Counter({'дискурс': 12,
         'дополнение': 83,
         'интерпретация': 234,
         'коррекция': 40,
         'модальность': 2,
         'обобщение': 51,
         'обр определение': 4,
         'определение': 81,
         'переспрос': 7,
         'переспрос инт': 49,
         'переспрос как': 7,
         'переформулирование': 109,
         'пример': 8,
         'следствие': 258,
         'спецификация': 55})

In [21]:
# Находим средние вектора
average = {}
for x, y in zip(tags, context):
    if x not in average.keys():
        average[x] = 0
    average[x] += y
for name, count in unique.items():
    average[name] /= count    

In [22]:
from pymystem3 import Mystem
m = Mystem()

In [23]:
# Это функция для парсинга предложения
def vectorise(item):
    item = re.sub('[Тт]о есть', 'тоесть', item)
    punctuation = (list(""",;:”$%^&*№()_—=+!...?|[]{}\"/<>`~±§«»°1234567890.""")
               + ['- ', ' -', ' - ', '― ', ' ―', ' ― ', " '", "' "])
    item = item.lower()
    for char in punctuation:
        item = item.replace(char, ' ')
    item = item.strip()
    lemmas = m.lemmatize(item)
    vector = 0
    for word in lemmas:
        if word != 'тоедать' and ' ' not in word:
            try:
                vector+=model.wv[word]
            except Exception as e:
                pass
    return vector

In [24]:
# Это функция для подсчета близости к типам значения
def find_type(sentence):
    vector = vectorise(sentence) # Превращаем предложение в вектор
    types = []
    for key in average.keys():
        types.append((key, cos(average[key], vector))) # Считаем близость нашего вектора и вектора-прототипа
    types = sorted(types, reverse=True, key=lambda x: float(x[1])) # Сортируем
    return types

## Оценка предсказательной силы

In [25]:
with open('examples.txt') as fh:
    lines = fh.readlines()

In [26]:
pprint(lines[:5])

['%дискурс\n',
 'А вот о возможностях фото и видео контента стоит рассказать немного '
 'подробнее. </s> <s> То есть в том понимании, как видит все это наша студия и '
 'ее сотрудники. !\n',
 'В этот день Якопо Бельбо глядел в глаза Истине … И это, то есть мгновение, '
 'когда он затормозил пространство и время !\n',
 'Раз предлагал даже зайти в ближайший салон связи, купить ей телефон и только '
 'оставить свой - то есть перезвонила бы если захотела .. !\n',
 '―\xa0Нам\xa0нужно,\xa0чтоб\xa0нас\xa0всех\xa0черти\xa0разорвали… То есть, '
 'черт возьми, я говорю, чтоб, тысяча чертей, тысяча залпов вам в задницу… '
 'черт!  [Митьки. Папуас из Гондураса (1987)] [омонимия не снята] !\n']


In [27]:
meanings = {}
for line in lines:
    if line.startswith('%'): # В файле идет название значения, а потом примеры на него
        line = line.strip()
        line = line.strip('%')
        key = line
        meanings[key] = []
    else:
        line = line.strip()
        line = line.replace(u'\xa0', ' ')
        line = re.sub('\[.+?\]', '', line)
        line = re.sub('\<.+?\>', '', line)
        meanings[key].append(line)

In [28]:
meanings['дискурс']

['А вот о возможностях фото и видео контента стоит рассказать немного подробнее.   То есть в том понимании, как видит все это наша студия и ее сотрудники. !',
 'В этот день Якопо Бельбо глядел в глаза Истине … И это, то есть мгновение, когда он затормозил пространство и время !',
 'Раз предлагал даже зайти в ближайший салон связи, купить ей телефон и только оставить свой - то есть перезвонила бы если захотела .. !',
 '― Нам нужно, чтоб нас всех черти разорвали… То есть, черт возьми, я говорю, чтоб, тысяча чертей, тысяча залпов вам в задницу… черт!    !',
 'Правой рукой он обнял, тоже распотевшего ираскрасневшегося, кума Селивана и о чем-то горячо рассуждал с ним. Сердце у бабы замерло: «запил» мелькнуло у нее в голове, и ноги ее подкосились.  ― То есть… я тебе говорю… во всякое время… и больше ничего…   ',
 '― И не «сливайтесь»!  То есть, я вам скажу!..   Он ухватился обеими руками за голову. !',
 'Для вас.. для маменьки вашей…  То есть, господи, боже мой!   И снова начиналось целовани

In [29]:
median_prediction = []
for key, sentences in meanings.items():
    position = []
    for sentence in sentences:
        types = find_type(sentence) # Ищем ранжирование типов по каждому предложению
        for i in range(1, len(types)+1):
            if types[i-1][0] == key: # Запоминаем позицию нужного
                position.append(i)
    median_prediction.append((key, np.median(position))) # Собираем медианы предсказаний

Получается, что лучше всего программа ловит дискурсивные значения *то есть*, а не союзные

In [30]:
median_prediction = sorted(median_prediction, key=lambda x: x[1])
median_prediction

[('коррекция', 1.0),
 ('переспрос', 2.0),
 ('следствие', 3.0),
 ('определение', 3.0),
 ('переспрос инт', 4.0),
 ('дополнение', 4.0),
 ('переформулирование', 4.0),
 ('дискурс', 4.0),
 ('интерпретация', 6.0),
 ('пример', 6.5),
 ('спецификация', 7.0),
 ('обобщение', 8.0),
 ('переспрос как', 12.0)]

## Попробуем сравнить ожидания и реальность...

In [31]:
sentence = 'И здесь соблюдалась полная демократия, то есть картину отбирал по своему вкусу очередной дежурный '

Это был пример следствия

In [32]:
find_type(sentence)

[('дополнение', 0.49646387),
 ('переформулирование', 0.49464464),
 ('следствие', 0.47175238),
 ('переспрос инт', 0.44966534),
 ('спецификация', 0.43783745),
 ('интерпретация', 0.428271),
 ('пример', 0.42404312),
 ('обобщение', 0.41845536),
 ('коррекция', 0.38692376),
 ('определение', 0.3796479),
 ('обр определение', 0.3568918),
 ('дискурс', 0.30305752),
 ('переспрос', 0.2543023),
 ('модальность', 0.24200524),
 ('переспрос как', 0.16896102)]

In [33]:
sentence = 'Очень, очень, очень желает с тобой познакомиться! — Да с какой же стати очень-то? — То есть не то чтобы... видишь, в последнее время, вот как ты заболел, мне часто и много приходилось об тебе поминать... Ну, он слушал... и как узнал, что ты по юридическому и кончить курса не можешь, по обстоятельствам, то сказал: «Как жаль!» Я и заключил... то есть всё это вместе, не одно ведь это; вчера Заметов...'

А вот тут да, дискурсивное + коррекция

In [34]:
find_type(sentence)

[('коррекция', 0.8345981),
 ('переспрос', 0.66444075),
 ('модальность', 0.649335),
 ('дискурс', 0.60710835),
 ('переспрос инт', 0.5093532),
 ('интерпретация', 0.4326337),
 ('следствие', 0.40162003),
 ('дополнение', 0.38091367),
 ('переспрос как', 0.36725906),
 ('переформулирование', 0.32998458),
 ('пример', 0.31459954),
 ('спецификация', 0.3097917),
 ('обр определение', 0.23680076),
 ('обобщение', 0.23121443),
 ('определение', 0.171176)]

In [35]:
sentence = 'Не дальше, как сегодня, то есть в субботу, опять со статьей произошло что-то'

Дополнение

In [36]:
find_type(sentence)

[('коррекция', 0.5870863),
 ('модальность', 0.3988665),
 ('переспрос', 0.36593),
 ('дискурс', 0.29356867),
 ('переспрос инт', 0.28157365),
 ('интерпретация', 0.23318017),
 ('дополнение', 0.2053265),
 ('переспрос как', 0.19805361),
 ('следствие', 0.19701575),
 ('переформулирование', 0.16128781),
 ('пример', 0.15274444),
 ('спецификация', 0.13818322),
 ('обобщение', 0.08339409),
 ('обр определение', 0.048011523),
 ('определение', 0.039526027)]

In [37]:
sentence = 'Он вникал в глубину этого сравнения и разбирал, что такое другие и что он сам, в какой степени возможна и справедлива эта параллель и как тяжела обида, нанесенная ему Захаром; наконец, сознательно ли оскорбил его Захар, то есть убежден ли он был, что Илья Ильич всё равно, что «другой», или так это сорвалось у него с языка, без участия головы'

Интерпретация

In [38]:
find_type(sentence)

[('коррекция', 0.6849089),
 ('переспрос инт', 0.5766381),
 ('модальность', 0.54515266),
 ('переспрос', 0.5402034),
 ('дополнение', 0.53942215),
 ('следствие', 0.5211004),
 ('интерпретация', 0.5183997),
 ('переформулирование', 0.4998029),
 ('дискурс', 0.49539855),
 ('спецификация', 0.4515638),
 ('пример', 0.44867957),
 ('обобщение', 0.40390095),
 ('обр определение', 0.37540218),
 ('определение', 0.3723401),
 ('переспрос как', 0.2135355)]

In [39]:
sentence = 'Недоверчив, скептик, циник... надувать любит, то есть не надувать, а дурачить...'

Коррекция

In [40]:
find_type(sentence)

[('коррекция', 0.61111975),
 ('переспрос', 0.52047914),
 ('переспрос как', 0.4836254),
 ('дискурс', 0.47677955),
 ('модальность', 0.42416152),
 ('переспрос инт', 0.3421536),
 ('интерпретация', 0.29362494),
 ('следствие', 0.2685734),
 ('дополнение', 0.25317886),
 ('спецификация', 0.21385588),
 ('переформулирование', 0.20698303),
 ('пример', 0.18612926),
 ('обобщение', 0.1367163),
 ('обр определение', 0.12662798),
 ('определение', 0.07190362)]

In [41]:
sentence = '― Я вечный двигатель изобрел.  ― То есть как? ― спросил Аркадий Максимович Фетисов. '

Переспрос как

In [42]:
find_type(sentence)

[('коррекция', 0.44792897),
 ('переспрос как', 0.38327762),
 ('переспрос', 0.3653465),
 ('дискурс', 0.32223785),
 ('модальность', 0.31283477),
 ('переспрос инт', 0.20357259),
 ('дополнение', 0.16124123),
 ('интерпретация', 0.14511849),
 ('спецификация', 0.1148869),
 ('следствие', 0.10626988),
 ('переформулирование', 0.07470271),
 ('пример', 0.061679058),
 ('обобщение', 0.030053873),
 ('обр определение', 0.023682129),
 ('определение', -0.005514154)]

In [43]:
sentence = '― У меня было плохое настроение.   ― То есть?   ― Мне показалось, что все уже было… '

Переспрос

In [44]:
find_type(sentence)

[('коррекция', 0.8154006),
 ('модальность', 0.6904732),
 ('переспрос', 0.6542083),
 ('дискурс', 0.5524111),
 ('переспрос инт', 0.45652768),
 ('интерпретация', 0.40393516),
 ('переспрос как', 0.3833468),
 ('следствие', 0.35151908),
 ('дополнение', 0.29680344),
 ('пример', 0.259354),
 ('спецификация', 0.2400244),
 ('переформулирование', 0.23602137),
 ('обр определение', 0.20430605),
 ('обобщение', 0.14350344),
 ('определение', 0.09449407)]

# Классификатор

Попробуем построить более сложный классификатор из встроенных в sklearn

In [45]:
import pandas as pd

In [46]:
# Создадим словарь объектов и ответов
dic = {}
dic['context'] = [i[1] for i in sim]
dic['type'] = tags

In [47]:
# Сделаем датафрейм
df = pd.DataFrame(dic)

In [48]:
X = df['context'].to_numpy().reshape(-1, 1) # Множество объектов
y = df['type'].to_numpy().reshape((-1, 1)) # Множество ответов

In [49]:
X[:2]

array([[0.36319348],
       [0.21303096]])

Создадим отложенную выборку для оценки качества модели

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Начнем работать с моделями

**KNN**

In [52]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [54]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='cosine')
classifier.fit(list(X_train), y_train)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [55]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.264

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
grid_params = {
    'n_neighbors' : [3, 5, 7],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan', 'cosine']
}

In [58]:
gs = GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    verbose=1,
    cv=3,
    n_jobs=-1
    )

In [59]:
gs_model = gs.fit(list(X_train), y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    4.0s finished
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

In [60]:
gs_model.best_params_

{'metric': 'cosine', 'n_neighbors': 5, 'weights': 'uniform'}

In [61]:
y_pred = gs_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.264

**Random Forest**

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [63]:
rnd_clf = RandomForestClassifier(random_state=42)

In [64]:
rnd_clf.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [65]:
y_pred = rnd_clf.predict(X_test)

In [66]:
accuracy_score(y_test, y_pred)

0.152

In [67]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [68]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_model = rf_random.fit(list(X_train), y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.3min finished
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [69]:
y_pred = rf_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.208

**LogReg**

In [70]:
from sklearn.linear_model import LogisticRegression

In [71]:
log_clf = LogisticRegression(random_state=42)

In [72]:
log_clf.fit(list(X_train), y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
y_pred = log_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.248